# Transcending Scales

In [1]:
from brian2 import *

## Single Neuron Transfer Property
### Single Neuron Current Injection

In [2]:
start_scope() #Ignore brian objects created prior to this command
# neuronal parameters
N = 1 # total number of neurons
vth = -55*mV# threshold potential
vr = -70*mV# reset potential
tau = 20.0*ms # membrane time constant
R = 4e7*ohm

In [3]:
#Equations are assigned as strings
eqs_neurons='''
inp : amp
dv/dt = -v/tau + inp*R : volt (unless refractory)
'''
#volt here represents the unit of the variable defined by the equation

In [ ]:
P=NeuronGroup(N=N,model=eqs_neurons,threshold='v>=vth',reset='v=vr',refractory=2*ms,method='euler')

P.v = vth*uniform(size=N)#initial value
P.inp = uniform(size=N) * 40*mV #input